### Load dataset

In [ ]:
import pandas as pd
from enum import StrEnum, auto
RANDOM_STATE = 42

# NOTE: StrEnum requires Python 3.11+
#       Can refactor to CONSTANTS instead
class FeatureVariant(StrEnum):
    LITERATURE = auto()
    RESEARCH = auto()
    STATISTICAL = auto()
    AUTOMATED = auto()

# Can be replaced with desired variant for different feature sets
GENE_FILE_VARIANT = FeatureVariant.RESEARCH

FILE_PATH = f"../Data/patient_genes_{GENE_FILE_VARIANT}.csv"

variant = 'svm'
df = pd.read_csv(FILE_PATH)

df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 977 entries, 0 to 976
Data columns (total 32 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   FYN      977 non-null    int64 
 1   BRCA1    977 non-null    int64 
 2   FOXC1    977 non-null    int64 
 3   TBC1D1   977 non-null    int64 
 4   LAG3     977 non-null    int64 
 5   CDK6     977 non-null    int64 
 6   GATA3    977 non-null    int64 
 7   CCND1    977 non-null    int64 
 8   PRR4     977 non-null    int64 
 9   EPCAM    977 non-null    int64 
 10  CD274    977 non-null    int64 
 11  PIK3CA   977 non-null    int64 
 12  TOP2A    977 non-null    int64 
 13  DCLK1    977 non-null    int64 
 14  MYC      977 non-null    int64 
 15  LRPPRC   977 non-null    int64 
 16  BRCA2    977 non-null    int64 
 17  TP53     977 non-null    int64 
 18  MKI67    977 non-null    int64 
 19  TTN      977 non-null    int64 
 20  CTLA4    977 non-null    int64 
 21  PTEN     977 non-null    int64 
 22  YE

### Imports

In [62]:
from sklearn.svm import SVC

%run "DataHelpers.ipynb"

### Dataset split: training and test data

In [63]:
X, y, X_train, X_test, y_train, y_test, test_case_ids = split_data(df, "tnbc", True)
print("\nApplied Smote")
X_smote, y_smote, X_train_smote, X_test_smote, y_train_smote, y_test_smote, test_case_ids_smote = split_data_apply_smote(df, "tnbc")

X_train.shape=(781, 30)
X_test.shape=(196, 30)
y_train.shape=(781,)
y_test.shape=(196,)

Applied Smote
X_train.shape=(1379, 30)
X_test.shape=(345, 30)
y_train.shape=(1379,)
y_test.shape=(345,)


### Support Vector Machine (SVM)

In [64]:
# Create model

model = SVC(random_state=RANDOM_STATE, probability=True)

def run_model(X_train: pd.DataFrame, X_test: pd.DataFrame, y_train: pd.Series, y_test: pd.Series, test_case_ids: pd.Series, is_smote: bool):
    # Train the model
    model.fit(X_train, y_train)

    # Model predictions
    y_pred = model.predict(X_test)
    y_prob = model.predict_proba(X_test)[:, 1]  # For ROC curves etc.

    # Save it in a dataframe, to CSV
    predictions = pd.DataFrame({
        "case_id": test_case_ids,
        "y_test": y_test,
        "y_pred": y_pred,
        "y_prob": y_prob
    })
    predictions.to_csv(f"../Data/model_output_{variant}_{GENE_FILE_VARIANT}_{'smote' if is_smote else ''}.csv", index=False)

    return y_pred, y_prob

In [ ]:
y_pred, y_prod = run_model(X_train, X_test, y_train, y_test, test_case_ids, False)
print_evaluated_model_accuracy(y_test, y_pred)

145    523E24A2-51B9-4658-BE2F-42E5FCCEBB17
36     378778d2-b331-4867-a93b-c64028c8b4c7
635    33cea538-4c3f-470a-972d-257c43241ab4
16     a5b44d66-c162-46b5-9df2-86305f0385c5
46     4a16429b-884f-4c95-8683-feeedf0d6a55
                       ...                 
518    d3d4d709-ab34-4617-aa8c-51a3d0a3efe6
113    9DDC014D-9A8B-407B-845F-452A73F08E5E
4      6623FC5E-00BE-4476-967A-CBD55F676EA6
832    8631969d-4ac6-4fe1-9db3-d47db604494a
636    03ab1e9f-4918-404d-b178-3a4fd929c5e8
Name: case_id, Length: 196, dtype: object
Accuracy: 0.94


## Smote applied

In [121]:
y_pred_smote, y_prod_smote = run_model(X_train_smote, X_test_smote, y_train_smote, y_test_smote, test_case_ids_smote, True)

print_evaluated_model_accuracy(y_test_smote, y_pred_smote)

Accuracy: 0.94


## Model cross validation

In [122]:
def run_cross_validation(X: pd.DataFrame, y: pd.Series, y_test: pd.Series, y_pred: pd.Series, y_prob: pd.Series, is_smote: bool) -> pd.DataFrame:
    metrics: pd.DataFrame = get_cross_validation_metrics(model, X, y, cv=5)
    test_metrics = get_metrics(y_test, y_pred, y_prob)
    test_metrics["fold"] = 0 # Initial test metrics (before cross validation)
    test = pd.DataFrame([test_metrics])
    test.set_index("fold", inplace=True)

    print_validated_model_accuracy(model, metrics)

    # Prepend test_metrics to metrics dataframe, export and display
    metrics = pd.concat([test, metrics])
    metrics.to_csv(f"../Data/model_metrics_{variant}_{GENE_FILE_VARIANT}_{'smote' if is_smote else ''}.csv", index=False)
    return metrics

In [123]:
# Still getting warning:
#   ConvergenceWarning: lbfgs failed to converge (status=1):
#                       STOP: TOTAL NO. OF F,G EVALUATIONS EXCEEDS LIMIT.
metrics = run_cross_validation(X, y, y_test, y_pred, y_prod, False)
metrics

Model validation for SVC:
[0.9285714285714286, 0.9285714285714286, 0.9384615384615385, 0.9230769230769231, 0.9128205128205128]

Mean accuracy: 0.9263



,accuracy,recall,precision,f1_score,roc_auc,true_positive,true_negative,false_positive,false_negative
fold,,,,,,,,,
0,0.943878,0.652174,0.833333,0.731707,0.975873,15,170,3,8
1,0.928571,0.608696,0.736842,0.666667,0.971852,14,168,5,9
2,0.928571,0.608696,0.736842,0.666667,0.966826,14,168,5,9
3,0.938462,0.565217,0.866667,0.684211,0.962588,13,170,2,10
4,0.923077,0.565217,0.722222,0.634146,0.966127,13,167,5,10
5,0.912821,0.521739,0.666667,0.585366,0.929474,12,166,6,11


In [124]:
# Still getting warning:
#   ConvergenceWarning: lbfgs failed to converge (status=1):
#                       STOP: TOTAL NO. OF F,G EVALUATIONS EXCEEDS LIMIT.
metric_smote = run_cross_validation(X_smote, y_smote, y_test_smote, y_pred_smote, y_prod_smote, True)
metric_smote

Model validation for SVC:
[0.9333333333333333, 0.9333333333333333, 0.927536231884058, 0.936231884057971, 0.9302325581395349]

Mean accuracy: 0.9321



,accuracy,recall,precision,f1_score,roc_auc,true_positive,true_negative,false_positive,false_negative
fold,,,,,,,,,
0,0.939130,0.970930,0.912568,0.940845,0.976912,167,157,16,5
1,0.933333,0.988372,0.890052,0.936639,0.967939,170,152,21,2
2,0.933333,0.970930,0.902703,0.935574,0.970561,167,155,18,5
3,0.927536,0.942197,0.915730,0.928775,0.972947,163,157,15,10
4,0.936232,0.953757,0.921788,0.937500,0.966192,165,158,14,8
5,0.930233,0.976744,0.893617,0.933333,0.971877,168,152,20,4
